In [1]:
import tensorflow as tf 

import numpy as np
import pandas as pd
import os
import time
import csv
print(tf.__version__)

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)


2.1.0
Name: /physical_device:GPU:0   Type: GPU


In [2]:
# np.set_printoptions(precision=3, suppress=True)

# feature_labels = np.genfromtxt('UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv', delimiter=',')
# train_file_path = tf.keras.utils.get_file("UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv", )
# data = np.genfromtxt('UNSW_2018_IoT_Botnet_Dataset_62.csv', delimiter=',')

base_dir = 'G:/Deel Learning Project/Entire Dataset'    

feature_labels = pd.read_csv("UNSW_2018_IoT_Botnet_Dataset_Feature_Names.csv", sep=',')

files = []
frame = pd.DataFrame()

#set range to 74 for all files (memory issue)
for i in range(1, 2):
    files.append(pd.read_csv(f"UNSW_2018_IoT_Botnet_Dataset_{i}.csv", sep=',', header=None))

frame = frame.append(files)

# for i in range(1, 74):
#     pd.read_csv(f"UNSW_2018_IoT_Botnet_Dataset_{i}.csv", sep=',', header=None).to_csv('temp.csv', mode='a', header=None)
    
# frame = pd.read_csv("temp.csv", sep=',', header=None)

frame.columns = feature_labels.columns

# print(labels)
print(frame.head())
print(frame.shape)
print(frame.columns)



g:\anaconda3\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   pkSeqID         stime flgs proto            saddr  sport           daddr  \
0        1  1.526344e+09    e   arp    192.168.100.1    NaN   192.168.100.3   
1        2  1.526344e+09    e   tcp    192.168.100.7    139   192.168.100.4   
2        3  1.526344e+09    e   udp  192.168.100.149  51838  27.124.125.250   
3        4  1.526344e+09    e   arp    192.168.100.4    NaN   192.168.100.7   
4        5  1.526344e+09    e   udp   192.168.100.27  58999   192.168.100.1   

   dport  pkts  bytes  ... spkts  dpkts  sbytes  dbytes       rate     srate  \
0    NaN     4    240  ...     2      2     120     120   0.002508  0.000836   
1  36390    10    680  ...     5      5     350     330   0.006190  0.002751   
2    123     2    180  ...     1      1      90      90  20.590960  0.000000   
3    NaN    10    510  ...     5      5     210     300   0.006189  0.002751   
4     53     4    630  ...     2      2     174     456   0.005264  0.001755   

      drate  attack  category  subcategory  

In [3]:
#Drop Seq ID
frame = frame.drop(['pkSeqID'], axis = 1)
# feature_labels.pop('pkSeqId')

#Drop Ips
frame = frame.drop(['saddr', 'daddr'], axis = 1)

#Drop blank fields
frame = frame.drop(['smac', 'dmac', 'soui', 'doui', 'sco', 'dco'], axis = 1)

#Drop flags and ports because they contain NaNs
frame = frame.drop(['flgs', 'sport', 'dport'], axis = 1)

print(frame.head())

          stime proto  pkts  bytes state         ltime  seq          dur  \
0  1.526344e+09   arp     4    240   CON  1.526345e+09    9  1195.996582   
1  1.526344e+09   tcp    10    680   CON  1.526346e+09   10  1453.945923   
2  1.526344e+09   udp     2    180   CON  1.526344e+09   11     0.048565   
3  1.526344e+09   arp    10    510   CON  1.526346e+09   12  1454.080322   
4  1.526344e+09   udp     4    630   CON  1.526345e+09   14   569.933960   

       mean    stddev  ...  spkts  dpkts  sbytes  dbytes       rate     srate  \
0  0.000006  0.000002  ...      2      2     120     120   0.002508  0.000836   
1  0.000028  0.000008  ...      5      5     350     330   0.006190  0.002751   
2  0.048565  0.000000  ...      1      1      90      90  20.590960  0.000000   
3  0.000238  0.000022  ...      5      5     210     300   0.006189  0.002751   
4  0.098505  0.080150  ...      2      2     174     456   0.005264  0.001755   

      drate  attack  category  subcategory  
0  0.000836

In [4]:
#Encode protocol

# frame = pd.concat([frame ,pd.get_dummies(frame['proto'], prefix='proto',dummy_na=False)],axis=1).drop(['proto'],axis=1)
# print(frame.head())

In [5]:
#Encode state

# frame = pd.concat([frame ,pd.get_dummies(frame['state'], prefix='state',dummy_na=False)],axis=1).drop(['state'],axis=1)
# print(frame.head())

In [6]:
#Encode category, skip if using pd.Categorical

# frame = pd.concat([frame ,pd.get_dummies(frame['category'], prefix='category',dummy_na=False)],axis=1).drop(['category'],axis=1)

# print(frame.head())

In [7]:
#Make the target the 'attack' column (binary classification)
def label_target(row):
    if row['attack'] == 1:
        return 1
    return 0

# frame['target'] = frame.apply(lambda row: label_target(row), axis = 1)
frame = frame.drop(['attack'], axis = 1)

#Make the tartget column a categorical version of the 'category' column (classification)

frame['target'] = pd.Categorical(frame['category'])

print("Categories:\n")
for i,cat in enumerate(frame['target'].cat.categories):
    print(i, cat)

frame['target'] = frame.target.cat.codes.astype(int)

frame = frame.drop(['category'], axis = 1)

#Remove the encoded categories and keep the target
# frame = frame[frame.columns.drop(list(frame.filter(regex='category_')))]

print(frame.head())

Categories:

0 Normal
1 Reconnaissance
          stime proto  pkts  bytes state         ltime  seq          dur  \
0  1.526344e+09   arp     4    240   CON  1.526345e+09    9  1195.996582   
1  1.526344e+09   tcp    10    680   CON  1.526346e+09   10  1453.945923   
2  1.526344e+09   udp     2    180   CON  1.526344e+09   11     0.048565   
3  1.526344e+09   arp    10    510   CON  1.526346e+09   12  1454.080322   
4  1.526344e+09   udp     4    630   CON  1.526345e+09   14   569.933960   

       mean    stddev  ...       max  spkts  dpkts  sbytes  dbytes       rate  \
0  0.000006  0.000002  ...  0.000007      2      2     120     120   0.002508   
1  0.000028  0.000008  ...  0.000042      5      5     350     330   0.006190   
2  0.048565  0.000000  ...  0.048565      1      1      90      90  20.590960   
3  0.000238  0.000022  ...  0.000261      5      5     210     300   0.006189   
4  0.098505  0.080150  ...  0.178655      2      2     174     456   0.005264   

      srate     d

In [8]:
#Encode subcategory

# frame = pd.concat([frame ,pd.get_dummies(frame['subcategory'], prefix='subcategory',dummy_na=False)],axis=1).drop(['subcategory'],axis=1)


#Remove subcategory
frame = frame.drop(['subcategory'], axis = 1)

print(frame.head())

          stime proto  pkts  bytes state         ltime  seq          dur  \
0  1.526344e+09   arp     4    240   CON  1.526345e+09    9  1195.996582   
1  1.526344e+09   tcp    10    680   CON  1.526346e+09   10  1453.945923   
2  1.526344e+09   udp     2    180   CON  1.526344e+09   11     0.048565   
3  1.526344e+09   arp    10    510   CON  1.526346e+09   12  1454.080322   
4  1.526344e+09   udp     4    630   CON  1.526345e+09   14   569.933960   

       mean    stddev  ...       min       max  spkts  dpkts  sbytes  dbytes  \
0  0.000006  0.000002  ...  0.000004  0.000007      2      2     120     120   
1  0.000028  0.000008  ...  0.000022  0.000042      5      5     350     330   
2  0.048565  0.000000  ...  0.048565  0.048565      1      1      90      90   
3  0.000238  0.000022  ...  0.000199  0.000261      5      5     210     300   
4  0.098505  0.080150  ...  0.018356  0.178655      2      2     174     456   

        rate     srate     drate  target  
0   0.002508  0.000

In [9]:
#Normalize 

cols_to_normalize = ['stime', 'ltime', 'dur', 'mean', 'stddev',
       'sum', 'min', 'max', 'rate', 'srate', 'drate']

# cols_to_normalize = frame.columns

print(frame.dtypes)

#Mean min-max
#frame=(df-df.min())/(df.max()-df.min())
frame[cols_to_normalize] = frame[cols_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#Mean normalization
# normalized_df=(df-df.mean())/df.std()

# normalized_frame = (frame-frame.min())/(frame.max()-frame.min())
# frame[cols_to_normalize] = frame[cols_to_normalize].apply(lambda x: (x - x.mean()) / x.std()))

print(frame.head())

stime     float64
proto      object
pkts        int64
bytes       int64
state      object
ltime     float64
seq         int64
dur       float64
mean      float64
stddev    float64
sum       float64
min       float64
max       float64
spkts       int64
dpkts       int64
sbytes      int64
dbytes      int64
rate      float64
srate     float64
drate     float64
target      int32
dtype: object
      stime proto  pkts  bytes state     ltime  seq       dur      mean  \
0  0.000148   arp     4    240   CON  0.001800    9  0.616220  0.000001   
1  0.000316   tcp    10    680   CON  0.002394   10  0.749125  0.000006   
2  0.000323   udp     2    180   CON  0.000000   11  0.000025  0.009774   
3  0.000325   arp    10    510   CON  0.002403   12  0.749194  0.000048   
4  0.000448   udp     4    630   CON  0.001066   14  0.293650  0.019824   

         stddev  ...           min       max  spkts  dpkts  sbytes  dbytes  \
0  8.001584e-07  ...  8.075216e-07  0.000001      2      2     120     120   
1

In [10]:
# frame = frame.astype('float64')

# print(frame.dtypes)

print(frame.head())

      stime proto  pkts  bytes state     ltime  seq       dur      mean  \
0  0.000148   arp     4    240   CON  0.001800    9  0.616220  0.000001   
1  0.000316   tcp    10    680   CON  0.002394   10  0.749125  0.000006   
2  0.000323   udp     2    180   CON  0.000000   11  0.000025  0.009774   
3  0.000325   arp    10    510   CON  0.002403   12  0.749194  0.000048   
4  0.000448   udp     4    630   CON  0.001066   14  0.293650  0.019824   

         stddev  ...           min       max  spkts  dpkts  sbytes  dbytes  \
0  8.001584e-07  ...  8.075216e-07  0.000001      2      2     120     120   
1  3.200634e-06  ...  4.441369e-06  0.000008      5      5     350     330   
2  0.000000e+00  ...  9.804321e-03  0.009713      1      1      90      90   
3  8.801743e-06  ...  4.017420e-05  0.000052      5      5     210     300   
4  3.206635e-02  ...  3.705717e-03  0.035731      2      2     174     456   

           rate         srate         drate  target  
0  2.006400e-09  8.360000e

In [11]:
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

train, test = train_test_split(frame, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

640000 train examples
160000 validation examples
200000 test examples


In [12]:
from tensorflow import feature_column


feature_columns = []
frame.pop('target')

# numeric cols
for header in frame.columns:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# print(train_ds)
# print(val_ds)
# print(test_ds)

In [ ]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Train for 20000 steps, validate for 5000 steps
Epoch 1/5


In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)